In [1]:
#pip install sqlite-fts4
#pip install import-ipynb

In [2]:
from ipynb.fs.full.Functions_sec import *
import import_ipynb
import fuzzymatcher as fm 
import pandas as pd
import os
pd.set_option('display.max_rows', 1200)

In [3]:
directory = os.getcwd()
path = directory + '/Data/'

# Reading files

In [4]:
Holdings=pd.read_csv(path + 'Holdings.csv')
Matched=pd.read_csv(path + 'Matched_v4.csv', sep=';')
Filings=pd.read_csv(path + 'Filings_df.csv')

# Nodes and Relationships Extraction

In [5]:
Nodes=Nodes_(Holdings,Filings)
duplicates_company_name=Nodes[1]
Nodes=Nodes[0]

In [6]:
Relationships=Relationships_(Holdings)

In [7]:
Relationships[(Relationships['value']==0)]

,cusip,cik_Filing,titleOfClass,putCall,CIK,investmentDiscretion,value,nameOfIssuer_unchanged
22,00214Q104,1074272,INNOVATION ETF,NaN,1697748,DFND,0,ARK Investment Management LLC
244,01609W102,1055964,SPONSORED ADS,NaN,1577552,SOLE,0,ALIBABA GROUP HLDG LTD
611,01609W102,1569765,SPONSORED ADS,NaN,1577552,SOLE,0,ALIBABA GROUP HLDG LTD
748,01609W102,1649647,SPONSORED ADS,Call,1577552,SOLE,0,ALIBABA GROUP HLDG LTD
1010,01609W102,1802473,SPONSORED ADS,NaN,1577552,SOLE,0,ALIBABA GROUP HLDG LTD
...,...,...,...,...,...,...,...,...
1817755,M8737E108,312069,COM,NaN,906338,SOLE,0,TARO PHARMACEUTICAL D
1817819,N71542109,312069,COM,NaN,161294,SOLE,0,PROQR RAPEUTICS NV
1817826,N96ESC012,919192,COM,NaN,1137861,SOLE,0,WRIGHT MEDICAL GROUP INC
1817832,Q6519V120,1742998,COM,NaN,1707919,SOLE,0,CENNTRO ELECTRIC GRP


In [8]:
Relationships=Relationships[~(Relationships['value']==0)]

In [9]:
Relationships=Relationships.groupby(by=['cik_Filing','CIK','titleOfClass','putCall','investmentDiscretion'],dropna=False).sum()
Relationships=Relationships.reset_index()

In [10]:
Relationships.to_csv(path + 'Relationships.csv', index=None)
Nodes.to_csv( path + "Nodes.csv" ,index=None)

## Validation of nodes

In [11]:
Relationships[~Relationships["cik_Filing"].isin (Nodes["cik"])]["cik_Filing"].unique()

array([], dtype=object)

In [12]:
Relationships[~Relationships["CIK"].isin (Nodes["cik"])]["CIK"].unique()

array([], dtype=object)

In [13]:
len(Relationships[~Relationships["CIK"].isin (Nodes["cik"])]["CIK"].unique())
# add these 3 

len(Relationships[~Relationships["cik_Filing"].isin (Nodes["cik"])]["cik_Filing"].unique())
# add these 6

0

## Notional value of derivative options

In [14]:
#different relationships for call and put, means different labels
Relationships_options=Relationships[~Relationships["putCall"].isna()]
#concat and then unique
a=pd.DataFrame(Relationships_options["CIK"].unique())
b=pd.DataFrame(Relationships_options["cik_Filing"].unique())
h=pd.concat((a,b),axis=0)
h.columns=["cik"]

In [15]:
h=pd.DataFrame(h["cik"].unique())
h.columns=["cik"]

In [16]:
Nodes_options=pd.merge(h,Nodes, how="left")

In [17]:
Relationships_options.to_csv(path + 'Relationships_options.csv')
Nodes_options.to_csv(path + 'Nodes_options.csv')

## Without derivative options

In [18]:
Relationships_wo_options=Relationships[Relationships["putCall"].isna()]
#concat and then unique
a=pd.DataFrame(Relationships_wo_options["CIK"].unique())
b=pd.DataFrame(Relationships_wo_options["cik_Filing"].unique())
h=pd.concat((a,b),axis=0)
h.columns=["cik"]

In [19]:
h=pd.DataFrame(h["cik"].unique())
h.columns=["cik"]

In [20]:
Nodes_wo_options=pd.merge(h,Nodes, how="left")

In [21]:
Relationships_wo_options.to_csv(path + 'Relationships_wo_options.csv')
Nodes_wo_options.to_csv( path +'Nodes_wo_options.csv')

## Relationships with self loops

In [22]:
len(Relationships[(Relationships["cik_Filing"])==(Relationships["CIK"])]) #check if self loops are between banks only

89

In [28]:
Self_loops=Relationships[(Relationships["cik_Filing"])==(Relationships["CIK"])]
self_loops_=pd.merge(Self_loops, Nodes, left_on="CIK", right_on="cik")

In [33]:
self_loops_

,cik_Filing,CIK,titleOfClass,putCall,investmentDiscretion,value,companyName,cik
0,1000275,1000275,COM,Call,DFND,2123000,ROYAL BK OF CANADA,1000275
1,1000275,1000275,COM,Put,SOLE,23882000,ROYAL BK OF CANADA,1000275
2,1000275,1000275,COM,NaN,DFND,8054463000,ROYAL BK OF CANADA,1000275
3,1000275,1000275,COM,NaN,OTR,185000,ROYAL BK OF CANADA,1000275
4,1000275,1000275,COM,NaN,SOLE,89000,ROYAL BK OF CANADA,1000275
5,1001085,1001085,COMMON STOCK,NaN,DFND,28000,BROOKFIELD ASSET MANAGEMENT IN,1001085
6,1001085,1001085,COMMON STOCK,NaN,SOLE,7869681000,BROOKFIELD ASSET MANAGEMENT IN,1001085
7,1005817,1005817,EQUITIES/EXCHANGE TRADED FUNDS,NaN,DFND,220000,TOMPKINS FINANCIAL CORP,1005817
8,1005817,1005817,EQUITIES/EXCHANGE TRADED FUNDS,NaN,SOLE,20548000,TOMPKINS FINANCIAL CORP,1005817
9,1013272,1013272,Common Stock,NaN,DFND,4861000,NORWOOD FINANCIAL CORP,1013272


In [ ]:
Filings[Filings['cik']==804328]

In [ ]:
Holdings[Holdings['cik_Filing']==804328]